In [ ]:
tech_colors= {
    'Transmission lines': "grey",
    'Battery storage' : '#80c944' ,
        'battery charger' : '#80c944' ,
    'battery discharger' : 'darkgreen' , 
    'battery' : 'darkgreen' ,        
    'Home battery storage' : '#ace37f'  ,
        'home battery charger' : '#ace37f'  ,
    'home battery discharger' : '#ace37f' ,    
    'home battery' : '#ace37f' ,    
    'H2 Fuel Cell': '#c251ae',
    'H2 Electrolysis': '#ff29d9',
    'H2 pipeline' : 'plum',
    'SMR': '#870c71',
    'SMR CC': '#4f1745',
    'Sabatier' : '#4f1745',
    'helmeth' : '#4f1745',
    'co2':'linen',
    'co2 vent':'linen',
    'co2 stored':'linen',
    'Electricity distribution grid': 'darkgrey',
    'electricity distribution grid': 'darkgrey',
    'distribution grid': 'darkgrey',
    'uranium': '#ff8c00',
    "onwind": "#235ebc",
    "Onshore wind": "#235ebc",
    "onshore wind": "#235ebc",
    'offwind': "#6895dd",
    'offwind-ac': "#6895dd",
    'Offshore wind': "#6895dd",
    'offshore wind': "#6895dd",
    'offshore wind ac': "#6895dd",
    'offwind-dc': "#74c6f2",
    'offshore wind dc': "#74c6f2",
    "Hydroelectricity": "#08ad97",
    "hydro": "#08ad97",
    "hydro+PHS": "#08ad97",
    "PHS": "#08ad97",
    "Pumped hydro storage": "teal",
    "hydro reservoir": "#08ad97",
    'hydroelectricity': '#08ad97',
    "ror": "#4adbc8",
    "Run of river": "#4adbc8",
    'Solar': "#f9d002",
    'Solar utility': "#f9d002",
    'solar': "#f9d002",
    'solar PV': "#f9d002",
    'Solar rooftop' : 'orange',
    'Solar distributed' : 'orange',
    'solar rooftop' : 'orange',
    'solar thermal': '#ffef60',
    'biomass': '#0c6013',
    'biomass chp': '#0c6013',    
    'solid biomass': '#06540d',
    'biogas': '#23932d',
    'waste': '#68896b',
    'geothermal': '#ba91b1',
    "OCGT": "#d35050",
    "gas": "#d35050",
        "CCGT": "#d35050",
    "natural gas": "#d35050",
    "CCGT": "#b20101",
    "Nuclear": "rosybrown",
        "nuclear": "rosybrown",
    "coal": "#707070",
    "lignite": "#9e5a01",
    "oil": "#262626",
    "H2 storage": "#ea048a",
    "hydrogen storage": "#ea048a",
    "battery": "#b8ea04",
    "Electric load": "#f9d002",
    "electricity": "#f9d002",
    "lines": "#70af1d",
    "transmission lines": "#70af1d",
    "AC-AC": "#70af1d",
    "AC line": "#70af1d",
    "links": "#8a1caf",
    "HVDC links": "#8a1caf",
    "DC-DC": "#8a1caf",
    "DC link": "#8a1caf",
    'Solar thermal': '#ffef60',
    "Oil and gas to power/heat":"#d35050",
    "Coal": "#707070",
    "Power to liquid": '#4f1745',
    "Power to gas":'#870c71',
    "Power to heat":'maroon',
    "Biomass":'#06540d',
    "Biogas":'#23932d',
    "BEV charger":'rebeccapurple',
    "V2G":'mediumpurple',
    "DAC":'lightcoral',
    "Heat storage":"brown",
     "gas CHP": "maroon",
     "biomass CHP": "darkolivegreen",
     "resistive heater": "indianred",
     "air heat pump": "chocolate",
     "ground heat pump": "sandybrown",
    "gas boiler": "crimson",        
        'industry' : 'black',
    'industry gas' : 'darkred',
    'industry biomass' : 'olive',
        "Fischer-Tropsch":"grey",
        'biogas to gas':'greenyellow',
        "heat pump":'darkslategrey',
    "resistive heater":'maroon',
    "gas for power" : "red"



}

In [ ]:
opt_name = {
    "Store": "e",
    "Line": "s",
    "Transformer": "s"
}
def calculate_costs(n, label, costs):

    for c in n.iterate_components(n.branch_components|n.controllable_one_port_components^{"Load"}):
        capital_costs = c.df.capital_cost*c.df[opt_name.get(c.name,"p") + "_nom_opt"]
        capital_costs_grouped = capital_costs.groupby(c.df.carrier).sum()

        capital_costs_grouped = pd.concat([capital_costs_grouped], keys=["capital"])
        capital_costs_grouped = pd.concat([capital_costs_grouped], keys=[c.list_name])

        costs = costs.reindex(capital_costs_grouped.index.union(costs.index))

        costs.loc[capital_costs_grouped.index, label] = capital_costs_grouped

        if c.name == "Link":
            p = c.pnl.p0.multiply(n.snapshot_weightings.generators, axis=0).sum()
        elif c.name == "Line":
            continue
        elif c.name == "StorageUnit":
            p_all = c.pnl.p.multiply(n.snapshot_weightings.generators, axis=0)
            p_all[p_all < 0.] = 0.
            p = p_all.sum()
        else:
            p = c.pnl.p.multiply(n.snapshot_weightings.generators, axis=0).sum()

        #correct sequestration cost
        if c.name == "Store":
            items = c.df.index[(c.df.carrier == "co2 stored") & (c.df.marginal_cost <= -100.)]
            c.df.loc[items, "marginal_cost"] = -20.

        marginal_costs = p*c.df.marginal_cost

        marginal_costs_grouped = marginal_costs.groupby(c.df.carrier).sum()

        marginal_costs_grouped = pd.concat([marginal_costs_grouped], keys=["marginal"])
        marginal_costs_grouped = pd.concat([marginal_costs_grouped], keys=[c.list_name])

        costs = costs.reindex(marginal_costs_grouped.index.union(costs.index))

        costs.loc[marginal_costs_grouped.index,label] = marginal_costs_grouped

    # add back in all hydro
    #costs.loc[("storage_units", "capital", "hydro"),label] = (0.01)*2e6*n.storage_units.loc[n.storage_units.group=="hydro", "p_nom"].sum()
    #costs.loc[("storage_units", "capital", "PHS"),label] = (0.01)*2e6*n.storage_units.loc[n.storage_units.group=="PHS", "p_nom"].sum()
    #costs.loc[("generators", "capital", "ror"),label] = (0.02)*3e6*n.generators.loc[n.generators.group=="ror", "p_nom"].sum()

    return costs

In [ ]:
import pandas as pd
import numpy as np
import pypsa
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches


main_scen={'../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':
          ['-nohomegen',''],
          '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':
          ['-nohomegen','']}   #scenario C


#main_scen={'../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist1-noH2network-higheff0.93{}_2030.nc':
 #          ['-nohomegen','-highrooftop6']}  #scenario D : scneario C- High rooftop : 7% loss+ 500 €/kW costs
                                         # for 'nohomegen' : doesn't matter waht rooftop potential is
   
   
transmission_expansion=[1.0] #,'opt']
years = [2030] 





In [ ]:
results=pd.DataFrame({})
scen_costs={}
costs=pd.DataFrame({})   #empty ,TODO: change name

for scen in main_scen:
    for i in range (2):
 #for tr_ex in transmission_expansion: 
    #for year in years:
             scenario = scen.format(main_scen[scen][i])
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                ,pd.Series({'Heat store [Gwh]' : n.stores.e_nom_opt.filter(like='water').sum()/1e3})
                                 ,pd.Series({'Total-cost [bn€]' : n.objective/1e09})
                                ,pd.Series({'Peak load [GW]' : n.loads_t.p[[i for i in n.loads.index if 'electricity' in n.loads.carrier[i]]].sum(axis=1).max()/1e3})])
             results[scenario]=result.groupby(result.index).sum() 
                                       #'SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                      #             'gas','H2 pipeline','uranium','coal' ,'lignite','oil',
                                       #            'DC'])  # 'ror','OCGT','H2 Electrolysis'
             scen_costs[scenario] = (calculate_costs(n,'capital+marginal',costs).sum(axis=1)).groupby(level=2).sum().div(1e9)  #in billiion Euros
             scen_costs[scenario] = scen_costs[scenario].rename(index={
                          'battery':'Battery storage','battery charger':'Battery storage','battery discharger':'Battery storage',
                          'home battery':'Home battery storage','home battery charger':'Home battery storage','home battery discharger':'Home battery storage',                                                       
                           'offwind-ac':'Offshore wind','offwind-dc':'Offshore wind', 'onwind':'Onshore wind',
                          'AC':'Transmission lines'  , 'DC':'Transmission lines','ror': 'Run of river',
                          'hydro':'Hydroelectricity' , 'nuclear':'Nuclear','uranium':'Nuclear',
                          'solar':'Solar utility', 'solar rooftop':'Solar rooftop',
                          'electricity distribution grid' : 'Electricity distribution grid',
                           'PHS' : 'Pumped hydro storage', 'H2':'H2 storage',
                           'SMR':'Power to gas'  , 'SMR CC':'Power to gas','oil': 'Oil and gas to power/heat',
                           'Fischer-Tropsch':'Power to liquid', 
                           'Sabatier':'Power to gas', 'helmeth':'Power to gas', 'coal' : 'Coal', 'lignite' :'Coal',
                          })  # co2 vent','co2','co2 stored',
             for ix in scen_costs[scenario].index :   
                if 'resistive heater' in ix or 'heat pump' in ix :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix : 'Power to heat',})
                if  any(w in ix for w in ['gas', 'OCGT', 'CCGT'])  :
                    scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Oil and gas to power/heat',})
                if 'biomass' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biomass',})
                if 'solar thermal' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Solar thermal',})
                if 'water tank' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Heat storage',})
                if 'biogas' in ix :scen_costs[scenario] =scen_costs[scenario].rename(index={ix:'Biogas',})
                #if 'biogas' in ix :scen_costs[scenario].rename(index={ix:'Biomass',})
                    
             scen_costs[scenario]=scen_costs[scenario].groupby(scen_costs[scenario].index).sum() 

In [ ]:
costs=pd.DataFrame({})

preferred_order = pd.Index(
    [
        "Transmission lines",
        "Electricity distribution grid",
        "Oil and gas to power/heat",
        "Coal",
        "Power to liquid",
        "Power to gas",
        "Biomass",
        "Biogas",
        "BEV charger",
        "V2G",
        "DAC",
        "Power to heat",
        "Heat storage",
        "H2 storage",
        "H2 pipeline",
        "H2 Electrolysis",
        "H2 Fuel Cell",
        "Pumped hydro storage",
        "Hydroelectricity",
        "Run of river",
        "Nuclear",
        "Onshore wind",
        "Offshore wind",
        "Battery storage",
        "Home battery storage",
        "Solar thermal",
        "Solar utility",
        "Solar rooftop",
    ]
)


for  scenario in reversed(scen_costs.keys()) :
    costs[scenario]=pd.DataFrame(scen_costs[scenario].drop(labels =['co2 vent','co2','co2 stored'
                                                                   ]))
                                                                  #  ,'oil','uranium','lignite','coal','gas' ]) ) 
                                                        #   ['SMR', 'SMR CC','Sabatier','co2 vent','co2','co2 stored',
                                                   #    'helmeth', 'oil','uranium','lignite','coal','gas', 'CCGT','OCGT'
                                                  #  ]) ) #dropping all the variables not visible in figure
    

order = preferred_order.intersection(costs[scenario].index).append(
        costs[scenario].index.difference(preferred_order))    
costs=costs.reindex(order)   
costs=costs.drop(labels =['process emissions','process emissions CC','Li ion'])
costs=costs.fillna(0)
results=results.fillna(0)


In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots(1, 2, figsize=(8,8)) #, sharey=True)
fig.subplots_adjust(hspace=0.1)
fig.subplots_adjust(wspace=1)

bar_width=0.3
index=np.arange(4) 

scenarios_description=[ 'Scenario C', 'Scenario D']

for i,base in enumerate(main_scen):
            scenario=base.format(main_scen[base][0]) 
            scenario_with=base.format(main_scen[base][1]) 
            bottom_p=0
            bottom_n=0
            print(scenario+' added:')
            axs[i].grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(costs[scenario].index):
              difference= costs[scenario_with][component]-costs[scenario][component]
              if difference > 0 :
                print(component, difference)    
                axs[i].bar(index[0], difference , bar_width, 
                           bottom=bottom_p,
                           color=tech_colors[component], zorder=3)
                bottom_p+=difference
              else:
                 axs[i].bar(index[0], difference , bar_width, 
                           bottom=bottom_n,
                           color=tech_colors[component], zorder=3)
                 bottom_n+=difference
                
            axs[i].axhline(y=0,lw=1.5, color='black',zorder=4)
            axs[i].set_xlabel("", rotation=0)
            axs[i].set_xticks([], [])
            axs[i].tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs[i].set_yticks(np.arange(0, 401, 100),fontsize=25)
            axs[i].set_ylabel(scenarios_description[i]+
                              '\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')
            
legend_manual=[]
for key in (costs[scenario].index):
  if key != 'V2G' and key != 'BEV charger' and key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1, 0.7),ncol=1,
           fontsize=14,frameon=False)



In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000


fig, axs = plt.subplots( figsize=(8,8)) #, sharey=True)
fig.subplots_adjust(hspace=0.07)
fig.subplots_adjust(wspace=0.5)

bar_width=0.3
index=np.arange(4) 

costs = costs.rename(index={
                          'Solar rooftop':'Solar distributed',})


for i,base in enumerate(main_scen):
    for j, category in enumerate(main_scen[base]):
            scenario=base.format(category) 
            bottom=0
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(costs[scenario].index):
                axs.bar(index[j], costs[scenario][component] , bar_width, bottom=bottom,
                           color=tech_colors[component], zorder=3)
                bottom+=costs[scenario][component]
            axs.set_xlabel("", rotation=0)
            axs.set_xticks([], [])
            axs.tick_params(axis='y', labelsize=18)
            #axs.set_facecolor("whitesmoke")
            #axs.set_yticks(np.arange(0, 401, 100),fontsize=25)
            if i==0 : axs.set_title('scenario D: High DPV potential, 7% loss 500 €/kW\nwithout distributed                      with distributed\n    generation                                    generation\n',fontsize=20, fontname = 'Ubuntu')
            #if j==0 : axs.set_ylabel(scenarios_description[scenario][0]+'\n'+
             #   scenarios_description[scenario][2]+'\n\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')
            axs.set_ylabel('system cost [bn€/a]\n',fontsize=20, fontname = 'Ubuntu')
            
legend_manual=[]
for key in (costs[scenario].index):
  if key != 'V2G' and key != 'BEV charger' and key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1, 1),ncol=1,
           fontsize=20,frameon=False)

for i ,scen in enumerate (main_scen.keys()):
    fig.text(0.45, 0.15*(3-i), "- "+str(round(scen_costs[scen.format(main_scen[scen][0])].sum()-
                                              scen_costs[scen.format(main_scen[scen][1])].sum(),2))
         +" bn€\n "+str(round(100*((scen_costs[scen.format(main_scen[scen][1])].sum()/
                                     scen_costs[scen.format(main_scen[scen][0])].sum())-1),2))+"%", 
         fontsize=18)


def add_arrow(ax0, ax1, pos0, pos1, **arrow_kwargs):
    ax0tr = ax0.transData  # Axis 0 -> Display
    ax1tr = ax1.transData  # Axis 1 -> Display
    figtr = fig.transFigure.inverted()  # Display -> Figure
    ptB = figtr.transform(ax0tr.transform(pos0))
    ptE = figtr.transform(ax1tr.transform(pos1))
    arrow = FancyArrowPatch(
        ptB,
        ptE,
        transform=fig.transFigure,  # Place arrow in figure coord system
        **arrow_kwargs,
    )
    fig.patches.append(arrow)


arrow_style = dict(arrowstyle="simple", mutation_scale=15, ec=None)
add_arrow(axs, axs, (0.4, 0.4), (0.7, 0.4), fc="black", **arrow_style)
#add_arrow(axs[1], axs[1], (0.4, 0.5), (0.7, 0.5), fc="black", **arrow_style)
#add_arrow(axs[2], axs[2], (0.4, 0.5), (0.7, 0.5), fc="black", **arrow_style)

#axs[0].set_yticks(np.arange(0, 301, 50),fontsize=25)
#axs[1].set_yticks(np.arange(0, 301, 50),fontsize=25)
#xs[2].set_yticks(np.arange(0, 301, 50),fontsize=25)


In [ ]:
tech_colors= {
    'Transmission lines': "grey",
    'Battery storage' : '#80c944' ,
        'battery charger' : '#80c944' ,
    'battery discharger' : 'darkgreen' , 
    'battery' : 'darkgreen' ,        
    'Home battery storage' : '#ace37f'  ,
        'home battery charger' : '#ace37f'  ,
    'home battery discharger' : '#ace37f' ,    
    'home battery' : '#ace37f' ,    
    'H2 Fuel Cell': '#c251ae',
    'H2 Electrolysis': '#ff29d9',
    'H2 pipeline' : 'plum',
    'SMR': '#870c71',
    'SMR CC': '#4f1745',
    'Sabatier' : '#4f1745',
    'helmeth' : '#4f1745',
    'co2':'linen',
    'co2 vent':'linen',
    'co2 stored':'linen',
    'Electricity distribution grid': 'darkgrey',
    'electricity distribution grid': 'darkgrey',
    'distribution grid': 'darkgrey',
    'uranium': '#ff8c00',
    "onwind": "#235ebc",
    "Onshore wind": "#235ebc",
    "onshore wind": "#235ebc",
    'offwind': "#6895dd",
    'offwind-ac': "#6895dd",
    'Offshore wind': "#6895dd",
    'offshore wind': "#6895dd",
    'offshore wind ac': "#6895dd",
    'offwind-dc': "#74c6f2",
    'offshore wind dc': "#74c6f2",
    "Hydroelectricity": "#08ad97",
    "hydro": "#08ad97",
    "hydro+PHS": "#08ad97",
    "PHS": "#08ad97",
    "Pumped hydro storage": "teal",
    "hydro reservoir": "#08ad97",
    'hydroelectricity': '#08ad97',
    "ror": "#4adbc8",
    "Run of river": "#4adbc8",
    'Solar': "#f9d002",
    'Solar utility': "#f9d002",
    'solar': "#f9d002",
    'solar PV': "#f9d002",
    'Solar rooftop' : 'orange',
    'solar rooftop' : 'orange',
    'solar thermal': '#ffef60',
    'biomass': '#0c6013',
    'solid biomass': '#06540d',
    'biogas': '#23932d',
    'waste': '#68896b',
    'geothermal': '#ba91b1',
    "OCGT": "#d35050",
    "gas": "#d35050",
        "CCGT": "#d35050",
    "natural gas": "#d35050",
    "CCGT": "#b20101",
    "Nuclear": "rosybrown",
        "nuclear": "rosybrown",
    "coal": "#707070",
    "lignite": "#9e5a01",
    "oil": "#262626",
    "H2 storage": "#ea048a",
    "hydrogen storage": "#ea048a",
    "battery": "#b8ea04",
    "Electric load": "#f9d002",
    "electricity": "#f9d002",
    "lines": "#70af1d",
    "transmission lines": "#70af1d",
    "AC-AC": "#70af1d",
    "AC line": "#70af1d",
    "links": "#8a1caf",
    "HVDC links": "#8a1caf",
    "DC-DC": "#8a1caf",
    "DC link": "#8a1caf",
    'Solar thermal': '#ffef60',
    "Oil and gas to power/heat":"#d35050",
    "Coal": "#707070",
    "Power to liquid": '#4f1745',
    "Power to gas":'#870c71',
    "Power to heat":'maroon',
    "Biomass":'#06540d',
    "Biogas":'#23932d',
    "BEV charger":'rebeccapurple',
    "V2G":'mediumpurple',
    "DAC":'lightcoral',
    "Heat storage":"brown",
     "gas CHP": "maroon",
     "biomass CHP": "darkolivegreen",
    "biomass boiler": "olivedrab",
     "resistive heater": "indianred",
     "air heat pump": "chocolate",
     "ground heat pump": "sandybrown",
    "gas boiler": "crimson",        
     "gas CHP/boiler": "maroon",
     "biomass CHP/boiler": "darklivegreen",
     "air/ground heat pump": "chocolate",

}

In [ ]:
def get_generation(n, carriers=None):

    dispatch = n.generators_t.p 

    generators_gen = n.snapshot_weightings.generators @ (dispatch) #n.snapshot_weightings.generators

    generators_gen = (
        generators_gen.groupby([n.generators.bus.map(n.buses.location), n.generators.carrier])
        .sum()
        .unstack()
    )
    
    links_gen= (-1) *n.snapshot_weightings.generators @(n.links_t.p1)
    links_gen = (links_gen.groupby([n.links.bus1.map(n.buses.location), n.links.carrier])
        .sum()
        .unstack()
    )
    #for i in links_gen.index:
     #     if 'EU' not in i :  
      #          generation.loc[ ( i, "nuclear" ) ]=links_gen['nuclear'].loc[i]
    generation_ = [generators_gen, links_gen]
    generation = pd.concat(generation_)

    return generation.drop("EU", axis=0).div(1e6)  #.stack()  TWh



In [ ]:
from matplotlib.patches import FancyArrowPatch
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000
plt.rc('hatch', linewidth=0.5)


fig, axs = plt.subplots(figsize=(6,8)) #, sharey=True)
fig.subplots_adjust(hspace=0.07)
fig.subplots_adjust(wspace=0.5)

bar_width=0.3
index=np.arange(4) 

carriers = {
        "gas CHP": "Gas CHP",
        "biomass CHP": "Biomass CHP",
        "resistive heater": "Resistive heater",
        "air heat pump": "Air heat pump",
        "ground heat pump": "Ground heat pump",
        "gas boiler": "Gas boiler",        
        "solar thermal": "Solar thermal",
       "biomass boiler":'Biomass boiler'

    }

types = [
         "decentral",
         "rural",
         "urban central",]



results=pd.DataFrame({})    
costs=pd.DataFrame({})    
scen_costs={}
hatches=['','/']
heat_sources=['gas CHP' , 'gas boiler','biomass CHP', 'biomass boiler', 'air heat pump','ground heat pump', 'resistive heater', 'solar thermal']

for i, category in enumerate(main_scen.keys()):
     for j in range(2):
            scenario=category.format(main_scen[category][j])
            n=pypsa.Network(scenario)
            generation = get_generation(n)
            heat_sums=generation[[i for i in generation.columns if any(carrier in i for carrier in carriers.keys())]].sum()
            
            scen_costs[scenario] = (calculate_costs(n,'capital+marginal',costs).sum(axis=1)).groupby(level=2).sum().div(1e9)  #in billiion Euros
            result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost [bn€]' : n.objective/1e09})
                                ,pd.Series({'Peak load [GW]' : n.loads_t.p[[i for i in n.loads.index if 'electricity' in n.loads.carrier[i]]].sum(axis=1).max()/1e3})])
            results[scenario]=result.groupby(result.index).sum() 
            
            for ix in heat_sums.index :  
                for key in carriers.keys():
                    if key in ix :
                                heat_sums =heat_sums.rename(index={ix : key,})
            heat_sums=heat_sums.groupby(heat_sums.index).sum() 
            bottom=0
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(heat_sources):
                axs.bar(index[i]+0.3*j, heat_sums[component] , bar_width, bottom=bottom, color=tech_colors[component],
                            hatch=hatches[j], edgecolor='snow', zorder=3)   #component
                bottom+=heat_sums[component]
            
            axs.bar(index[i]+0.3*j, (-2)*n.links_t.p2.filter(like='Fuel Cell').sum().sum()/1e6
                    , bar_width, bottom=bottom, color='purple',
                            hatch=hatches[j], edgecolor='snow', zorder=3) 
            axs.set_xticks((np.arange(2))+bar_width*0.5)
            xlabels = ['default\nrooftop\npotential', 'high\nrooftop\npotential']
            axs.set_xticklabels(xlabels,fontsize=16, rotation=0,ha='center', rotation_mode='anchor')
            axs.tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            axs.set_yticks(np.arange(0, 4000, 500),fontsize=25)
            axs.set_ylabel('Scenario C'+'\n'+'Energy generation [TWh]\n',fontsize=18, fontname = 'Ubuntu')
            #if j==0 : axs[i].set_ylabel(scenarios_description[scenario][0]+'\n'+
             #   scenarios_description[scenario][2]+'\n\n'+'bn€/a',fontsize=20, fontname = 'Ubuntu')

                
legend_manual=[]
for key in (heat_sums.index):
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
    
plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1,1),ncol=1,
           fontsize=20,frameon=False)

##since the electricity distribution has higher power losses, the system moves towards direct heat sources

In [ ]:
import datetime
from matplotlib.dates import DateFormatter
fig, axs = plt.subplots(2, 1, figsize=(3,6),sharex=True, sharey=True) 
fig.subplots_adjust(wspace=0.1)

plt.rcParams['figure.dpi'] = 1000

colors=['navy','darkred']
colors_range=['ghostwhite','snow']  #'whitesmoke','snow'


date_form = DateFormatter("%b %-d")

for i, category in enumerate(main_scen.keys()):
  for j in range(2):
    scenario=category.format(main_scen[category][j])
    n=pypsa.Network(scenario)
    
    df_AC=pd.DataFrame(n.buses_t.marginal_price[[i for i in n.buses.index if 'heat' in n.buses.carrier[i]]]
                       .mean(axis=1) )  ##added
    df_AC['hour']=df_AC.index.hour

   ##added
    l_AC,= axs[i].plot(df_AC['hour'][0:12],df_AC.groupby(['hour']).mean(),color=colors[j],label='AC')
    
    axs[i].fill_between(df_AC['hour'][0:12],(df_AC.groupby(['hour']).mean()-df_AC.groupby(['hour']).std()).squeeze(),
                 (df_AC.groupby(['hour']).mean()+df_AC.groupby(['hour']).std()).squeeze(),color=colors_range[j],)
    r_AC = axs[i].fill(np.NaN, np.NaN, 'ghostwhite', alpha=1)
    axs[i].set_xlabel("hour of the day",fontsize=12)
    axs[i].tick_params(axis='y', labelsize=12)
    #axs[i].set_yticks(np.arange(0, 8.1, 2),fontsize=14)   
    axs[i].set_xticks(np.arange(0, 24, 2),fontsize=14)  
    #'if i==1 :  axs[i].legend([l_load, (l_dist, r_dist)], ["load (mean+range)", "HV to LV evergy\nflow (mean+range)"], loc=2)
    axs[i].set_ylabel('Average daily\nheat price [€/MWh]',fontsize=12,
                  fontname = 'Ubuntu',labelpad=10)

#fig.autofmt_xdate()

plt.legend([(r_AC[0],l_AC), (r_AC[0],l_AC)], ["without distribed\nsolar (mean+range)","with distribed\nsolar (mean+range)"], 
           loc='center', bbox_to_anchor=(1.8, 0.5),ncol=1,
          fontsize=12, frameon=False)
#plt.legend(loc='center right', bbox_to_anchor=(1.4, 0.5),ncol=1,
 #         fontsize=12, frameon=False) 

In [ ]:
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000
plt.rc('hatch', linewidth=0.5)

main_scen={'../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':
           ['-nohomegen',''],
           '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':
           ['-nohomegen',''],}
fig, axs = plt.subplots(figsize=(6,7)) #, sharey=True)
fig.subplots_adjust(hspace=0.07)
fig.subplots_adjust(wspace=0.5)

results=pd.DataFrame({})
scen_costs={}
costs=pd.DataFrame({})   #empty ,TODO: change name

for scen in main_scen.keys():
    for i in range (2):
 #for tr_ex in transmission_expansion: 
    #for year in years:
             scenario = scen.format(main_scen[scen][i])
             n=pypsa.Network(scenario)
             result = pd.concat([pd.Series(n.links.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series(n.generators.groupby("carrier").p_nom_opt.sum())
                                 ,pd.Series({'Total-cost [bn€]' : n.objective/1e09})
                                ,pd.Series({'Peak load [GW]' : n.loads_t.p[[i for i in n.loads.index if 'electricity' in n.loads.carrier[i]]].sum(axis=1).max()/1e3})])
             results[scenario]=result.groupby(result.index).sum() 
                                       #'SMR', 'SMR CC','Sabatier','co2 vent','helmeth',
                                      #             'gas','H2 pipeline','uranium','coal' ,'lignite','oil',
                                       #            'DC'])  # 'ror','OCGT','H2 Electrolysis'
bar_width=0.25
index=np.arange(4) 

results1=results
hatches=['','/']

EVS=['V2G', 'BEV charger']

for i, category in enumerate(main_scen.keys()):
 for j in range(2):
            scenario=category.format(main_scen[category][j]) 
            bottom=0
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(EVS):
                axs.bar(index[i]+0.25*j, results1[scenario][component]/1e6 , bar_width, bottom=bottom,   #TW
                           color=tech_colors[component], hatch=hatches[j], edgecolor='snow', zorder=3)
                bottom+=results1[scenario][component]/1e6
            axs.set_xticks((np.arange(2))+bar_width*0.5)
            xlabels = ['default\nrooftop\npotential', 'high\nrooftop\npotential']
            axs.set_xticklabels(xlabels,fontsize=16, rotation=0,ha='center', rotation_mode='anchor')
            axs.tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            axs.set_yticks(np.arange(0, 3, 0.4),fontsize=25)
            #if i==0 : axs.set_title('without distributed                      with distributed\n    generation                                    generation\n',fontsize=20, fontname = 'Ubuntu')
            axs.set_ylabel('Scenario C'+'\n'+'Installed capacity [TW]\n',fontsize=18, fontname = 'Ubuntu')
            
legend_manual=[]
for key in (EVS):
  if  key != 'Coal' :
    legend_manual.append(mpatches.Patch(color=tech_colors[key], label=key))
legend1=plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1.01, 0.8),ncol=1,
           fontsize=18,frameon=False)
plt.gca().add_artist(legend1)

legend_manual_2=[]
for i, label in enumerate(['without distributed solar', 'with distributed solar']):
    legend_manual_2.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', 
                                        label=label))
legend2=plt.legend(handles=legend_manual_2,loc='center right', bbox_to_anchor=(2.05, 0.9),ncol=1,
           fontsize=18, frameon=False) 
plt.gca().add_artist(legend2)

#axs[0].set_yticks(np.arange(0, 4.1, 0.5),fontsize=25)
#axs[1].set_yticks(np.arange(0, 4.1, 0.5),fontsize=25)
#axs.set_yticks(np.arange(0, 9.1, 2.25),fontsize=25)



In [ ]:
import matplotlib.font_manager

plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')
#plt.rcParams['figure.dpi'] = 1000
plt.rc('hatch', linewidth=0.5)


fig, axs = plt.subplots(figsize=(6,7)) #, sharey=True)
fig.subplots_adjust(hspace=0.07)
fig.subplots_adjust(wspace=0.5)

main_scen={'../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}_2030.nc':
           ['-nohomegen',''],
           '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network{}-highrooftop6_2030.nc':
           ['-nohomegen',''],}

bar_width=0.25
index=np.arange(4) 

results1=results
hatches=['','/']

EVS=['V2G generation', 'BEV charger generation']
EVS_=['V2G', 'BEV charger']

for i, category in enumerate(main_scen.keys()):
 for j in range(2):
            scenario=category.format(main_scen[category][j]) 
            n=pypsa.Network(scenario)
            bottom=0
            axs.grid(axis = 'y',color='gainsboro', zorder=0)
            for color, component in enumerate(EVS_):
                axs.bar(index[i]+0.25*j, 2*n.links_t.p0[[i for i in n.links.index if component in i ]].sum().sum()/1e6 #TWh
                        , bar_width, bottom=bottom,   #TW
                           color=tech_colors[component], hatch=hatches[j], edgecolor='snow', zorder=3)
                bottom+=n.links_t.p0[[i for i in n.links.index if component in i ]].sum().sum()/1e6
            axs.set_xticks((np.arange(2))+bar_width*0.5)
            xlabels = ['default\nrooftop\npotential', 'high\nrooftop\npotential']
            axs.set_xticklabels(xlabels,fontsize=16, rotation=0,ha='center', rotation_mode='anchor')
            axs.tick_params(axis='y', labelsize=18)
            #axs[i].set_facecolor("whitesmoke")
            #axs.set_yticks(np.arange(0, 3, 0.4),fontsize=25)
            #if i==0 : axs.set_title('without distributed                      with distributed\n    generation                                    generation\n',fontsize=20, fontname = 'Ubuntu')
            axs.set_ylabel('Scenario C'+'\n'+'Energy generation [TWh]\n',fontsize=18, fontname = 'Ubuntu')
            
legend_manual=[]
for color,key in enumerate(EVS):
    legend_manual.append(mpatches.Patch(color=tech_colors[EVS_[color]], label=key))
legend1=plt.legend(handles=legend_manual[::-1],loc='upper left', bbox_to_anchor=(1.01, 0.8),ncol=1,
           fontsize=18,frameon=False)
plt.gca().add_artist(legend1)

legend_manual_2=[]
for i, label in enumerate(['without distributed solar', 'with distributed solar']):
    legend_manual_2.append(mpatches.Patch(facecolor='whitesmoke', hatch=hatches[i], edgecolor='k', 
                                        label=label))
legend2=plt.legend(handles=legend_manual_2,loc='center right', bbox_to_anchor=(2.05, 0.9),ncol=1,
           fontsize=18, frameon=False) 
plt.gca().add_artist(legend2)

#axs[0].set_yticks(np.arange(0, 4.1, 0.5),fontsize=25)
#axs[1].set_yticks(np.arange(0, 4.1, 0.5),fontsize=25)
#axs.set_yticks(np.arange(0, 9.1, 2.25),fontsize=25)



In [ ]:
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(1, 2, figsize=(10,4),sharex=True, sharey=True) 
fig.subplots_adjust(wspace=0.1)

plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

country=' '

date_form = DateFormatter("%b %-d")

networks={
          '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-nohomegen-highrooftop6_2030.nc':'Without distributed generation',
              '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-highrooftop6_2030.nc':'With distributed generation',

}
for i,network in enumerate(networks.keys()):
     n=pypsa.Network(network)
     print(n.links_t.p0[[i for i in n.links.index if 'BEV charger' in i ]].sum(axis=1).sum()/1e3)
     print(n.links_t.p0[[i for i in n.links.index if 'V2G' in i]].sum(axis=1).sum()/1e3)
               
     axs[i].plot((2)*n.links_t.p0[[i for i in n.links.index if 'BEV charger' in i and 'industry' not in i]].filter
          (like=country).sum(axis=1)/1e3,color=tech_colors['BEV charger'],label='battery electric\nvehicles charger')   
    
     axs[i].plot(2*n.links_t.p0.filter(like='V2G').filter(like=country).sum(axis=1)/1e3,
       color=tech_colors['V2G'],label='vehicle to grid')    
    
     #axs[i].set_title('Scenario B, node '+country+'\n' +networks[network]+'\n',fontsize=14)
     axs[i].set_xlabel("day of the year",fontsize=12)
     axs[i].tick_params(axis='y', labelsize=12)
     #axs[i].set_yticks(np.arange(0, 6.1, 2),fontsize=14)   
     axs[i].xaxis.set_major_formatter(date_form)
   
axs[0].set_ylabel('scenario D\nEnergy generation [GWh]',fontsize=14, fontname = 'Ubuntu',labelpad=10)

#fig.autofmt_xdate()


plt.legend(loc='center right', bbox_to_anchor=(1.9, 0.5),ncol=1,
           fontsize=12, frameon=False) 

In [ ]:
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(1, 2, figsize=(10,4),sharex=True, sharey=True) 
fig.subplots_adjust(wspace=0.1)

plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

country=' '

date_form = DateFormatter("%b %-d")

networks={
          '../sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-nohomegen_2030.nc':'Without distributed generation',
              '../sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc':'With distributed generation',

}
for i,network in enumerate(networks.keys()):
     n=pypsa.Network(network)
     print(n.links_t.p0[[i for i in n.links.index if 'BEV charger' in i ]].sum(axis=1).sum()/1e3)
     print(n.links_t.p0[[i for i in n.links.index if 'BEV charger' in i ]].sum(axis=1).max()/1e3)
     print(n.links_t.p0[[i for i in n.links.index if 'V2G' in i]].sum(axis=1).sum()/1e3)

               
     axs[i].plot((2)*n.links_t.p0[[i for i in n.links.index if 'BEV charger' in i ]]
                 .sum(axis=1)/1e3,color=tech_colors['BEV charger'],label='battery electric\nvehicles charger')   
    
     axs[i].plot(2*n.links_t.p0.filter(like='V2G').sum(axis=1)/1e3,
       color=tech_colors['V2G'],label='vehicle to grid')  
        
     #axs[i].set_title('Scenario B, node '+country+'\n' +networks[network]+'\n',fontsize=14)
     axs[i].set_xlabel("day of the year",fontsize=12)
     axs[i].tick_params(axis='y', labelsize=12)
     #axs[i].set_yticks(np.arange(0, 6.1, 2),fontsize=14)   
     axs[i].xaxis.set_major_formatter(date_form)
   
axs[0].set_ylabel('scenario C\nEnergy generation [GWh]',fontsize=14, fontname = 'Ubuntu',labelpad=10)

#fig.autofmt_xdate()


plt.legend(loc='center right', bbox_to_anchor=(1.9, 0.5),ncol=1,
           fontsize=12, frameon=False) 

## Fourier power spectrum

In [ ]:
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(2, 2, figsize=(12,6),sharex=True, ) 
fig.subplots_adjust(wspace=0.1)

def spectrum1(h, dt=1):
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.array([1/f for f in np.arange(0,dt,1/(nt))])[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = (np.abs(ft)/nt) ** 2

    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw


plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

#networks={
#          '../sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-nohomegen_2030.nc':'Without distributed generation',
#              '../sector_coupled_181_newbase_highloss_noH2network/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc':'With distributed generation',
#
#}
networks={
          '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-nohomegen_2030.nc':'Without distributed generation',
              '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network_2030.nc':'With distributed generation',

}


colors=['lightcoral','navy']
countries=['ES1 8' , 'IT1 0', 'DE1 0', 'ES', ' ']
country=' '

date_form = DateFormatter("%b %-d")

techs=['distribution grid HV to LV', 'distribution grid LV to HV']
network_type=networks.values()
data = pd.DataFrame(index=pd.MultiIndex.from_product([pd.Series(data=techs, name='tech',),
                                                       pd.Series(data=network_type, name='network_type',)]), 
                      columns=pd.Series(data=np.arange(0,4380), name='hour',))
idx = pd.IndexSlice

for i,network in enumerate(networks.keys()):
  n=pypsa.Network(network)
  network_type= networks[network] 
        
  data.loc[idx['distribution grid HV to LV', network_type,], :]=np.array(2*n.links_t.p0.filter(like='distribution grid').filter(like=country)
              [n.links_t.p0.filter(like='distribution grid').filter(like=country)>0].sum(axis=1))
  data.loc[idx['distribution grid LV to HV', network_type,], :]=np.array((-2)*n.links_t.p0.filter(like='distribution grid').filter(like=country)
              [n.links_t.p0.filter(like='distribution grid').filter(like=country)<0].sum(axis=1))
  

max=data.max(axis=1).max()
data = data/max  #normalize to show difference in data size 
     
for i,network_type in enumerate(networks.values()):
   for j,tech in enumerate(techs):
        
     data_= data.loc[idx[tech, network_type,], : ] 
     data_.reset_index(drop = True,inplace=True)
     dt=1
     freqs1, ps1, psd1 = spectrum1(data_, dt=dt)
     axs[i,j].semilogx(freqs1, ps1, colors[j], )   # ps1/ps1.max()
    

     #axs[i].set_yticks(np.arange(0, 6.1, 2),fontsize=14)   
  
axs[0,0].sharey(axs[1,0])
   
axs[0,0].set_title('HV to LV',fontsize=14, fontname = 'Ubuntu')
axs[0,1].set_title('LV to LV',fontsize=14, fontname = 'Ubuntu')
axs[0,0].set_ylabel('Without distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)
axs[1,0].set_ylabel('With distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)

for ax in axs.flat:
    ax.set_xlabel('(hours)')
    
fig.text(0.5, 1 ,'Scenario C'
         ,fontsize=14, fontname = 'Ubuntu')

#fig.autofmt_xdate()



In [ ]:
#frequency for storage
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(2, 3, figsize=(15,4),sharex=True, ) 
fig.subplots_adjust(wspace=0.2)

def spectrum1(h, dt=1):
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.array([1/f for f in np.arange(0,dt,1/(nt))])[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = (np.abs(ft)/nt) ** 2

    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw


plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

# scenario A:
#networks={'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-nohomegen-higheff_2030.nc':'Without distributed generation',
 #        'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc':'With distributed generation',}



colors=['lightcoral','darkred','navy','darkblue','darkgreen',]
countries=['ES1 8' , 'IT1 0', 'DE1 0', 'ES', ' ']
country=' '

date_form = DateFormatter("%b %-d")


#techs=['battery charger', 'battery discharger','home battery charger','home battery discharger','Battery store']
techs=['battery discharger','home battery discharger','Battery store']

network_type=networks.values()
data = pd.DataFrame(index=pd.MultiIndex.from_product([pd.Series(data=techs, name='tech',),
                                                       pd.Series(data=network_type, name='network_type',)]), 
                      columns=pd.Series(data=np.arange(0,4380), name='hour',))
idx = pd.IndexSlice

for i,network in enumerate(networks.keys()):
  n=pypsa.Network(network)
  network_type= networks[network] 
  for tech in techs:
     if tech!='Battery store':
         data.loc[idx[tech, network_type,], :]=np.array(
            2*n.links_t.p0[[i for i in n.links.index if n.links.carrier[i]==tech]]
           .filter(like=country).sum(axis=1))
     else:
         data.loc[idx['Battery store', network_type,], :]=np.array(
                (2*n.stores_t.e.filter(like='battery').filter(like=country).sum(axis=1)/  #ALL batteries
                    n.stores.e_nom_opt.filter(like='battery').sum()))
  

     
for i,network_type in enumerate(networks.values()):
   for j,tech in enumerate(techs):
     #print(tech)     
     data.loc[idx[tech, network_type,], : ]=(data.loc[idx[tech, network_type,], : ]/
         data.loc[tech].max().max())  #normalize with/without for same maximimum

     data_= data.loc[idx[tech, network_type,], : ] 
     data_.reset_index(drop = True,inplace=True)
     dt=1
     freqs1, ps1, psd1 = spectrum1(data_, dt=dt)
     axs[i,j].semilogx(freqs1, ps1, colors[j], )   # ps1/ps1.max()
     axs[0,j].sharey(axs[1,j])
     if i==0: axs[i,j].set_title(tech+'\n',fontsize=14, fontname = 'Ubuntu')
  

#axs[0,0].set_title('Bateries',fontsize=14, fontname = 'Ubuntu')
#axs[0,1].set_title('Home batteries',fontsize=14, fontname = 'Ubuntu')
#axs[0,2].set_title('battery fill level',fontsize=14, fontname = 'Ubuntu')

axs[0,0].set_ylabel('Without distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)
axs[1,0].set_ylabel('With distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)

for ax in axs.flat:
    ax.set_xlabel('(hours)')
    
fig.text(0.25, 1 ,'                   Scenario C\n'
         ,fontsize=14, fontname = 'Ubuntu')

#fig.autofmt_xdate()



In [ ]:
#frequency for storage
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(2, 3, figsize=(15,4),sharex=True,) 
fig.subplots_adjust(wspace=0.2)


def spectrum1(h, dt=1):
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.array([1/f for f in np.arange(0,dt,1/(nt))])[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = (np.abs(ft)/nt) ** 2

    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw



plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

# scenario A:
#networks={'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-nohomegen-higheff_2030.nc':'Without distributed generation',
 #        'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc':'With distributed generation',}


colors=['darkred','navy','darkgreen',]
countries=['ES1 8' , 'IT1 0', 'DE1 0', 'ES', ' ']
country=' '

date_form = DateFormatter("%b %-d")


techs=['H2 Electrolysis','Fuel Cell','H2 store']
network_type=networks.values()
data = pd.DataFrame(index=pd.MultiIndex.from_product([pd.Series(data=techs, name='tech',),
                                                       pd.Series(data=network_type, name='network_type',)]), 
                      columns=pd.Series(data=np.arange(0,4380), name='hour',))
idx = pd.IndexSlice

for i,network in enumerate(networks.keys()):
  n=pypsa.Network(network)
  network_type= networks[network] 
  for tech in techs:
     if tech!='H2 store':
         data.loc[idx[tech, network_type,], :]=np.array(
           (2*n.links_t.p0.filter(like=tech)
           .filter(like=country).sum(axis=1)))
     else:
         data.loc[idx['H2 store', network_type,], :]=np.array(
         ((n.stores_t.e[n.stores[n.stores.carrier.isin(['H2'])].index].filter(like=country).sum(axis=1)*100/
         n.stores.e_nom_opt[n.stores[n.stores.carrier.isin(['H2'])].index].filter(like=country).sum()).squeeze()))
  

     
for i,network_type in enumerate(networks.values()):
   for j,tech in enumerate(techs):
     #print(tech)     
     data.loc[idx[tech, network_type,], : ]=(data.loc[idx[tech, network_type,], : ]/
         data.loc[tech].max().max())  #normalize with/without for same maximimum

     data_= data.loc[idx[tech, network_type,], : ] 
     data_.reset_index(drop = True,inplace=True)
     dt=1
     freqs1, ps1, psd1 = spectrum1(data_, dt=dt)
     axs[i,j].semilogx(freqs1, ps1, colors[j], )   # ps1/ps1.max()
     axs[0,j].sharey(axs[1,j])
     if i==0: axs[i,j].set_title(tech,fontsize=14, fontname = 'Ubuntu')
  




axs[0,0].set_ylabel('Without distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)
axs[1,0].set_ylabel('With distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)

for ax in axs.flat:
    ax.set_xlabel('(hours)')
    
fig.text(0.25, 1 ,'                   Scenario C'
         ,fontsize=14, fontname = 'Ubuntu')

#fig.autofmt_xdate()



In [ ]:
#frequency for storage
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(2, 3, figsize=(15,4),sharex=True,) 
fig.subplots_adjust(wspace=0.2)

def spectrum1(h, dt=1):
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.array([1/f for f in np.arange(0,dt,1/(nt))])[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = (np.abs(ft)/nt) ** 2

    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw



plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

# scenario A:
#networks={'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-nohomegen-higheff_2030.nc':'Without distributed generation',
 #        'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc':'With distributed generation',}

colors=['darkred','navy','darkgreen',]
countries=['ES1 8' , 'IT1 0', 'DE1 0', 'ES', ' ']
country=' '

date_form = DateFormatter("%b %-d")


techs=['PHS charge','PHS discharge','PHS store']
network_type=networks.values()
data = pd.DataFrame(index=pd.MultiIndex.from_product([pd.Series(data=techs, name='tech',),
                                                       pd.Series(data=network_type, name='network_type',)]), 
                      columns=pd.Series(data=np.arange(0,4380), name='hour',))
idx = pd.IndexSlice

for i,network in enumerate(networks.keys()):
  n=pypsa.Network(network)
  network_type= networks[network] 
  data.loc[idx['PHS charge', network_type,], : ]=np.array(
  ((2*n.storage_units_t.p.filter(like='PHS').filter(like=country)
              [n.storage_units_t.p.filter(like='PHS').filter(like=country)>0].sum(axis=1))/
           n.storage_units.p_nom_opt.filter(like='PHS').filter(like=country).sum(axis=0)) )
  
  data.loc[idx['PHS discharge', network_type,], : ]=np.array(
  (((-2)*n.storage_units_t.p.filter(like='PHS').filter(like=country)
              [n.storage_units_t.p.filter(like='PHS').filter(like=country)<0].sum(axis=1))/
           n.storage_units.p_nom_opt.filter(like='PHS').filter(like=country).sum(axis=0)) )

  data.loc[idx['PHS store', network_type,], : ]=np.array(
    ((2)*n.storage_units_t.state_of_charge.filter(like='PHS').filter(like=country).sum(axis=1)/
                   6*n.storage_units.p_nom.filter(like='PHS').filter(like=country).sum()))
     
for i,network_type in enumerate(networks.values()):
   for j,tech in enumerate(techs):
     #print(tech)     
     data.loc[idx[tech, network_type,], : ]=(data.loc[idx[tech, network_type,], : ]/
         data.loc[tech].max().max())  #normalize with/without for same maximimum

     data_= data.loc[idx[tech, network_type,], : ] 
     data_.reset_index(drop = True,inplace=True)
     dt=1
     freqs1, ps1, psd1 = spectrum1(data_, dt=dt)
     axs[i,j].semilogx(freqs1, ps1, colors[j], )   # ps1/ps1.max()
     axs[0,j].sharey(axs[1,j])
     if i==0: axs[i,j].set_title(tech,fontsize=14, fontname = 'Ubuntu')
  


axs[0,0].set_ylabel('Without distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)
axs[1,0].set_ylabel('With distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)

for ax in axs.flat:
    ax.set_xlabel('(hours)')
    
fig.text(0.25, 1 ,'                   Scenario C'
         ,fontsize=14, fontname = 'Ubuntu')

#fig.autofmt_xdate()



In [ ]:
#frequency for storage
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(2, 3, figsize=(15,4),sharex=True,) 
fig.subplots_adjust(wspace=0.2)

def spectrum1(h, dt=1):
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.array([1/f for f in np.arange(0,dt,1/(nt))])[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = (np.abs(ft)/nt) ** 2

    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw



plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

# scenario A:
#networks={'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-nohomegen-higheff_2030.nc':'Without distributed generation',
 #        'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc':'With distributed generation',}

colors=['darkred','navy','darkgreen',]
countries=['ES1 8' , 'IT1 0', 'DE1 0', 'ES', ' ']
country=' '

date_form = DateFormatter("%b %-d")


techs=['BEV charger','Vehicle to grid','EV battery']
network_type=networks.values()
data = pd.DataFrame(index=pd.MultiIndex.from_product([pd.Series(data=techs, name='tech',),
                                                       pd.Series(data=network_type, name='network_type',)]), 
                      columns=pd.Series(data=np.arange(0,4380), name='hour',))
idx = pd.IndexSlice

for i,network in enumerate(networks.keys()):
  n=pypsa.Network(network)
  network_type= networks[network] 
  data.loc[idx['BEV charger', network_type,], : ]=np.array(
      ((2*n.links_t.p0[[i for i in n.links.index if n.links.carrier[i]=='BEV charger']]
           .filter(like=country)).sum(axis=1)))
  
  data.loc[idx['Vehicle to grid', network_type,], : ]=np.array(
    ((2*n.links_t.p0[[i for i in n.links.index if n.links.carrier[i]=='V2G']]
           .filter(like=country)).sum(axis=1)))

  data.loc[idx['EV battery', network_type,], : ]=np.array(
       2*n.stores_t.e.filter(like='battery storage').filter(like=country).sum(axis=1)/  #ALL batteries
           n.stores.e_nom_opt.filter(like='battery storage').sum())  
     
        
for i,network_type in enumerate(networks.values()):
   for j,tech in enumerate(techs):
     #print(tech)     
     data.loc[idx[tech, network_type,], : ]=(data.loc[idx[tech, network_type,], : ]/
         data.loc[tech].max().max())  #normalize with/without for same maximimum

     data_= data.loc[idx[tech, network_type,], : ] 
     data_.reset_index(drop = True,inplace=True)
     dt=1
     freqs1, ps1, psd1 = spectrum1(data_, dt=dt)
     axs[i,j].semilogx(freqs1, ps1, colors[j], )   # ps1/ps1.max()
     axs[0,j].sharey(axs[1,j])
     if i==0: axs[i,j].set_title(tech,fontsize=14, fontname = 'Ubuntu')
  


axs[0,0].set_ylabel('Without distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)
axs[1,0].set_ylabel('With distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)

for ax in axs.flat:
    ax.set_xlabel('(hours)')
    
fig.text(0.25, 1 ,'                   Scenario C'
         ,fontsize=14, fontname = 'Ubuntu')

#fig.autofmt_xdate()



In [ ]:
#frequency for storage
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(2, 3, figsize=(15,4),sharex=True,) 
fig.subplots_adjust(wspace=0.2)

def spectrum1(h, dt=1):
    nt = len(h)
    npositive = nt//2
    pslice = slice(1, npositive)
    freqs = np.array([1/f for f in np.arange(0,dt,1/(nt))])[pslice] 
    ft = np.fft.fft(h)[pslice]
    psraw = (np.abs(ft)/nt) ** 2

    # Convert PS to Power Spectral Density
    psdraw = psraw * dt * nt  # nt * dt is record length
    return freqs, psraw, psdraw



plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

# scenario A:
#networks={'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-nohomegen-higheff_2030.nc':'Without distributed generation',
 #        'postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-solar+p3-dist1-higheff_2030.nc':'With distributed generation',}

colors=['darkred','navy','darkgreen',]
countries=['ES1 8' , 'IT1 0', 'DE1 0', 'ES', ' ']
country=' '

date_form = DateFormatter("%b %-d")


techs=['thermal storage charger','thermal storage discharger','thermal store fill level']
network_type=networks.values()
data = pd.DataFrame(index=pd.MultiIndex.from_product([pd.Series(data=techs, name='tech',),
                                                       pd.Series(data=network_type, name='network_type',)]), 
                      columns=pd.Series(data=np.arange(0,4380), name='hour',))
idx = pd.IndexSlice

for i,network in enumerate(networks.keys()):
  n=pypsa.Network(network)
  network_type= networks[network] 
  data.loc[idx['thermal storage charger', network_type,], : ]=np.array(
      ((2*n.links_t.p0[[i for i in n.links.index if 'water tanks charger' in n.links.carrier[i]]]
           .filter(like=country)).sum(axis=1)))
  
  data.loc[idx['thermal storage discharger', network_type,], : ]=np.array(
      ((2*n.links_t.p0[[i for i in n.links.index if 'water tanks discharger' in n.links.carrier[i]]]
           .filter(like=country)).sum(axis=1)))

  data.loc[idx['thermal store fill level', network_type,], : ]=np.array(
      (2*n.stores_t.e[[i for i in n.stores.index if 'water tank' in n.stores.carrier[i]]]  #all residential+services
           .filter(like=country).sum(axis=1)))
    


for i,network_type in enumerate(networks.values()):
   for j,tech in enumerate(techs):
     #print(tech)     
     data.loc[idx[tech, network_type,], : ]=(data.loc[idx[tech, network_type,], : ]/
         data.loc[tech].max().max())  #normalize with/without for same maximimum

     data_= data.loc[idx[tech, network_type,], : ] 
     data_.reset_index(drop = True,inplace=True)
     dt=1
     freqs1, ps1, psd1 = spectrum1(data_, dt=dt)
     axs[i,j].semilogx(freqs1, ps1, colors[j], )   # ps1/ps1.max()
     axs[0,j].sharey(axs[1,j])
     if i==0: axs[i,j].set_title(tech,fontsize=14, fontname = 'Ubuntu')
  


axs[0,0].set_ylabel('Without distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)
axs[1,0].set_ylabel('With distributed\n generation',fontsize=14, fontname = 'Ubuntu',labelpad=10)

for ax in axs.flat:
    ax.set_xlabel('(hours)')
    
fig.text(0.25, 1 ,'                   Scenario C'
         ,fontsize=14, fontname = 'Ubuntu')

#fig.autofmt_xdate()



## highrooftop heat+industry

In [ ]:
import datetime
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(1, 2, figsize=(6,2.5),sharex=True, sharey=True)   #10, 4
fig.subplots_adjust(wspace=0.1)

plt.rcParams['figure.dpi'] = 1000
plt.rc('font',family='Ubuntu')
plt.rc('axes',edgecolor='gainsboro')

colors=['lightcoral','navy']
countries=['ES1 8' , 'IT1 0', 'DE1 0', 'ES', ' ']
country=' '
networks={

          '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-nohomegen-highrooftop6_2030.nc':['-nohomegen',''],
              '../Stable-sector-coupled/postnetworks/elec_s_181_lv1.1__Co2L0p05-2H-T-H-B-I-solar+p3-dist2-noH2network-highrooftop6_2030.nc':['-nohomegen',''],

}
date_form = DateFormatter("%b") #%-d

for i,network in enumerate(networks.keys()):
     n=pypsa.Network(network)

     axs[i].plot(2*n.links_t.p0.filter(like='heat pump').filter(like=country).sum(axis=1)/1e3,   #GWh
       color='orange',)
       
     #axs[i].plot((2)*n.loads_t.p[[i for i in n.loads.index if 'heat' in i and 'industry' not in i]].filter
     #      (like=country).sum(axis=1)/1e3,color='navy',label='non-industrial heat demand')
        
     #axs[i].plot((2)*n.links_t.p0[[i for i in n.links.index if 'biomass' in i and 'industry' not in i]].filter
      #    (like=country).sum(axis=1)/1e3,color='black',label='biomass for indusrty')   
    
     #axs[i].plot((2)*n.links_t.p0[[i for i in n.links.index if 'gas' in i and 'industry 'in i]].filter
      #    (like=country).sum(axis=1)/1e3,color='red',label='gas for industry')      
    
     #axs[i].set_title('Scenario B, node '+country+'\n' +networks[network]+'\n',fontsize=14)
     axs[i].set_xlabel("day of the year",fontsize=12)
     axs[i].tick_params(axis='y', labelsize=12)
     axs[i].set_yticks(np.arange(0, 350, 100),fontsize=14)   
     axs[i].xaxis.set_major_formatter(date_form)
   
axs[0].set_ylabel('Heat pumps\nenergy generation [GWh]',fontsize=14, fontname = 'Ubuntu',labelpad=10)

#fig.autofmt_xdate()


#plt.legend(loc='center right', bbox_to_anchor=(1.7, 0.5),ncol=1,
 #          fontsize=12, frameon=False) 